In [37]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.cross_validation import train_test_split
import math
import easy_excel
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess


def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0)
seq=[]
m6a_2614_sequence="data/human.fasta"
path="data/"
outputname="human_jacakknife_PSNP"
classifier="SVM"
RNA_code='ACGU'
interval=1
select_num=40
division_num=2260
divided_num=division_num*1.0

fh=open(m6a_2614_sequence)
for line in fh:
    if line.startswith('>'):
        continue
    else:
        seq.append(line.replace('\n','').replace('\r',''))
fh.close()

def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError
positive_seq=seq[:len(seq)/2]
# X_train, X_test, y_train, y_test = cross_validation.train_test_split(train_data, train_target, test_size=0.1, random_state=0)
negative_seq=seq[len(seq)/2:]
kf = KFold(n_splits=1130,shuffle=False)  

y_pred_prob_all=[]
y_pred_all=[]
Y_all=[]
ACC_all=0
precision_all=0
recall_all=0
SN_all=0
SP_all=0
GM_all=0
TP_all=0
TN_all=0
FP_all=0
FN_all=0
F_measure_all=0
F1_Score_all=0
pos_all=0
neg_all=0
MCC_all=0
echo=0
for train_index , test_index in kf.split(positive_seq):  
    print "echo :",echo
    echo+=1
    positive_df=pd.DataFrame(positive_seq)
    positive_x_train=positive_df.iloc[train_index,:]
    positive_y_train=positive_df.iloc[test_index,:]
    negative_df=pd.DataFrame(negative_seq)
    negative_x_train=negative_df.iloc[train_index,:]
    negative_y_train=negative_df.iloc[test_index,:]
    positive_negative_x_train=pd.concat([positive_x_train,negative_x_train],axis=0)
    positive_negative_y_train=pd.concat([positive_y_train,negative_y_train],axis=0)

    code_values1=make_kmer_list(1,RNA_code)
    code_len1=len(code_values1)
    positive_seq_value1=[[0 for jj in xrange(len(seq[0]))] for ii in xrange(code_len1)]
    negative_seq_value1=[[0 for jj in xrange(len(seq[0]))] for ii in xrange(code_len1)]
    for i,line_value in enumerate(positive_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-1 :
                for p,c_value in enumerate(code_values1):
                    if c_value==line_value[0][j:j+1]:
                        positive_seq_value1[p][j]+=1
    positive_seq_value1=np.matrix(positive_seq_value1)*1.0/(len(seq)/2)
    for i,line_value in enumerate(negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-1 :
                for p,c_value in enumerate(code_values1):
                    if c_value==line_value[0][j:j+1]:
                        negative_seq_value1[p][j]+=1
    negative_seq_value1=np.matrix(negative_seq_value1)*1.0/(len(seq)/2)
    
    
    final_seq_value1=[[0 for ii in xrange(len(seq[0]))] for jj in xrange(len(positive_negative_x_train))]
    for i,line_value in enumerate(positive_negative_x_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-1 :
                for p,c_value in enumerate(code_values1):
                    if c_value==line_value[0][j:j+1]:
                          final_seq_value1[i][j]=positive_seq_value1[p,j]-negative_seq_value1[p,j]
    y_final_seq_value1=[[0 for ii in xrange(len(seq[0]))] for jj in xrange(len(positive_negative_y_train))]


    for i,line_value in enumerate(positive_negative_y_train.values):
        for j,code_value in enumerate(line_value[0]):
            if j<= len(line_value[0])-1 :
                for p,c_value in enumerate(code_values1):
                    if c_value==line_value[0][j:j+1]:
                          y_final_seq_value1[i][j]=positive_seq_value1[p,j]-negative_seq_value1[p,j]

    for jack_index,jack_data in enumerate(y_final_seq_value1):
        X_train1 = np.array(final_seq_value1)
        Y_train = list(map(lambda x: 1, xrange(len(X_train1) / 2)))
        Y2_train = list(map(lambda x: 0, xrange(len(X_train1) / 2)))
        Y_train.extend(Y2_train)
        Y_train = np.array(Y_train)
        
        if jack_index==0:
            Y_test=[0]
            X_test1 = np.array(y_final_seq_value1[1])
            X_train1=np.concatenate([X_train1,np.array(y_final_seq_value1[0]).reshape(1,41)],axis=0)
            Y_train=np.concatenate([Y_train.reshape(division_num-2,1),np.array([1]).reshape(1,1)],axis=0)
        else:
            Y_test=[1]
            X_test1 = np.array(y_final_seq_value1[0])
            X_train1=np.concatenate([X_train1,np.array(y_final_seq_value1[1]).reshape(1,41)],axis=0)
            Y_train=np.concatenate([Y_train.reshape(division_num-2,1),np.array(0).reshape(1,1)],axis=0)
        svc = svm.SVC(probability=True)
        parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
        clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
        Y_train=np.array(list(Y_train))
        Y_train1=list([1 if x==1 else 0 for x in Y_train ])
        clf.fit(X_train1, Y_train1)
        C=clf.best_params_['C']
        y_pred_prob=clf.predict_proba(X_test1)

        gamma=clf.best_params_['gamma']
        print 'c:',C,'gamma:',gamma
        y_pred=clf.predict(X_test1)

        y_pred_prob_all.extend(y_pred_prob)
        y_pred_all.extend(y_pred)
        Y_all.append(Y_test)
        
all_y=[np.array(Y_all).astype(int).T[0],np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv(path+outputname+"_"+classifier+"_jackknife_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T[0], list(np.array(y_pred_prob_all).astype(list)[:,1]))

roc_auc = metrics.roc_auc_score(np.array(Y_all).T[0],np.array(y_pred_prob_all).astype(list)[:,1])
ACC=metrics.accuracy_score(np.array(Y_all).T[0],list(np.array(y_pred_all).astype(int)))
print "ACC",ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int))) 
F1_Score=metrics.f1_score(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
pos=TP+FN
neg=FP+TN


savedata=[str(X_train1.shape[1]),u"正："+str(pos)+u'负：'+str(neg),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN,
            SP, GM,F_measure,F1_Score,MCC,roc_auc,TP, FN,FP,TN]
final_out_to_excel.append(savedata)
print savedata
pd.DataFrame(final_out_to_excel).to_excel(path+'jackknife_'+classifier+"_"+outputname+'.xlsx',sheet_name="_crossvalidation",header=None,index=None)

echo : 0
c: 14.2543794902 gamma: 4.0
ACC 1.0
ACC_all: 1.0
------------------------------------


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

c: 14.2543794902 gamma: 4.0
ACC 0.0
ACC_all: 1.0
------------------------------------
echo : 1
c: 14.2543794902 gamma: 4.0
ACC 1.0
ACC_all: 2.0
------------------------------------


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

c: 14.2543794902 gamma: 4.0
ACC 1.0
ACC_all: 3.0
------------------------------------
echo : 2
c: 14.2543794902 gamma: 4.0
ACC 1.0
ACC_all: 4.0
------------------------------------


...

c: 14.2543794902 gamma: 4.0
ACC 1.0
ACC_all: 2055.0
------------------------------------
echo : 1129
c: 14.2543794902 gamma: 4.0
ACC 1.0
ACC_all: 2056.0
------------------------------------


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


c: 14.2543794902 gamma: 4.0
ACC 1.0
ACC_all: 2057.0
------------------------------------


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: could not broadcast input array from shape (2260,1) into shape (2260)

In [52]:
all_y=[np.array(Y_all).astype(int),np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]

roc_auc = metrics.roc_auc_score(np.array(Y_all).T[0],np.array(y_pred_prob_all).astype(list)[:,1])
ACC=metrics.accuracy_score(np.array(Y_all).T[0],list(np.array(y_pred_all).astype(int)))
print "ACC",ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int))) 
F1_Score=metrics.f1_score(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
pos=TP+FN
neg=FP+TN


savedata=[str(X_train1.shape[1]),u"正："+str(pos_all)+u'负：'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN,
            SP, GM,F_measure,F1_Score,MCC,roc_auc,TP,
            FN,FP,TN]
final_out_to_excel.append(savedata)
print savedata

ACC 0.91017699115
['41', u'\u6b63\uff1a1130.0\u8d1f\uff1a1130.0', 'svmC:14.2543794902gamma:4.0', 0.91017699115044248, 1.0, 0.820353982300885, 0.820353982300885, 1.0, 0.9057339467530655, 0.90131259115216333, 0.90131259115216333, 0.8339207731937075, 0.91232124676951998, 927.0, 203.0, 0.0, 1130.0]


In [53]:
all_y=[np.array(Y_all).astype(int).T[0],np.array(y_pred_all).astype(int),np.array(y_pred_prob_all).astype(list)[:,1]]
pd.DataFrame(np.matrix(all_y).T).to_csv("jackknife_data/"+outputname+"_"+classifier+"_jackknife_predict.csv",header=None,index=False)
fpr, tpr, thresholds = roc_curve(np.array(Y_all).T[0], list(np.array(y_pred_prob_all).astype(list)[:,1]))

roc_auc = metrics.roc_auc_score(np.array(Y_all).T[0],np.array(y_pred_prob_all).astype(list)[:,1])
ACC=metrics.accuracy_score(np.array(Y_all).T[0],list(np.array(y_pred_all).astype(int)))
print "ACC",ACC
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int))) 
F1_Score=metrics.f1_score(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(np.array(Y_all).T[0], list(np.array(y_pred_all).astype(int)))
pos=TP+FN
neg=FP+TN


savedata=[str(X_train1.shape[1]),u"正："+str(pos_all)+u'负：'+str(neg_all),'svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN,
            SP, GM,F_measure,F1_Score,MCC,roc_auc,TP,
            FN,FP,TN]
final_out_to_excel.append(savedata)
print savedata
pd.DataFrame(final_out_to_excel).to_excel("jackknife_data/"+'jackknife_'+classifier+"_"+outputname+'.xlsx',sheet_name="_crossvalidation",header=None,index=None)

ACC 0.91017699115
['41', u'\u6b63\uff1a1130.0\u8d1f\uff1a1130.0', 'svmC:14.2543794902gamma:4.0', 0.91017699115044248, 1.0, 0.820353982300885, 0.820353982300885, 1.0, 0.9057339467530655, 0.90131259115216333, 0.90131259115216333, 0.8339207731937075, 0.91232124676951998, 927.0, 203.0, 0.0, 1130.0]


In [56]:
final_out_to_excel=[]
row0 = [u'特征集', u'样本个数', u'分类器', u'Accuracy', u'Precision', u'Recall', u'SN', u'SP',
                u'Gm', u'F_measure', u'F_score', u'MCC', u'ROC曲线面积', u'tp', u'fn', u'fp', u'tn']
final_out_to_excel.append(row0)
final_out_to_excel.append(savedata)
print savedata
pd.DataFrame(final_out_to_excel).to_excel("jackknife_data/"+'jackknife_'+classifier+"_"+outputname+'.xlsx',sheet_name="_crossvalidation",header=None,index=None)

['41', u'\u6b63\uff1a1130.0\u8d1f\uff1a1130.0', 'svmC:14.2543794902gamma:4.0', 0.91017699115044248, 1.0, 0.820353982300885, 0.820353982300885, 1.0, 0.9057339467530655, 0.90131259115216333, 0.90131259115216333, 0.8339207731937075, 0.91232124676951998, 927.0, 203.0, 0.0, 1130.0]
